In [16]:
import numpy as np
import pandas as pd
import random

import torch
import torch.nn as nn
import torch.nn.functional as F

import seaborn as sns
import matplotlib.pyplot as plt
import os
from datetime import date
import datetime

# Generating the training data for the Heat and Diffusion Model

In [17]:
data_dir = "./LakePIAB/"
depth_steps = 25

print(os.getcwd())

/home/robert/Projects/LakePIAB/MCL/01_data_processing


In [18]:
meterological_data_df = pd.read_csv("./../../output/py_meteorology_input.csv")
meterological_data_df = meterological_data_df[1:] # considering everything from 2nd time step

num_time_steps = meterological_data_df.shape[0]
depth_list = np.array(list(range(1, depth_steps+1)) * num_time_steps)
depth_df = pd.DataFrame(data={'depth':depth_list})

#repeating the dataframe depth_steps number of times
meterological_data_df = pd.DataFrame(np.repeat(meterological_data_df.values, depth_steps, axis=0), columns=meterological_data_df.columns)
meterological_data_df = pd.concat([depth_df, meterological_data_df], ignore_index=False, axis=1)
meterological_data_df

,depth,time,AirTemp_degC,Longwave_Wm-2,Latent_Wm-2,Sensible_Wm-2,Shortwave_Wm-2,lightExtinct_m-1,ShearVelocity_mS-1,ShearStress_Nm-2,Area_m2
0,1,2006-01-01 02:00:00,-1.220007,590.851292,-30.129462,-35.559056,0.0,0.8,1.757216,0.008038,36000000.0
1,2,2006-01-01 02:00:00,-1.220007,590.851292,-30.129462,-35.559056,0.0,0.8,1.757216,0.008038,36000000.0
2,3,2006-01-01 02:00:00,-1.220007,590.851292,-30.129462,-35.559056,0.0,0.8,1.757216,0.008038,36000000.0
3,4,2006-01-01 02:00:00,-1.220007,590.851292,-30.129462,-35.559056,0.0,0.8,1.757216,0.008038,36000000.0
4,5,2006-01-01 02:00:00,-1.220007,590.851292,-30.129462,-35.559056,0.0,0.8,1.757216,0.008038,36000000.0
...,...,...,...,...,...,...,...,...,...,...,...
3065970,21,2019-12-29 00:00:00,3.86001,597.535984,25.682372,36.65838,0.0,0.8,6.709571,0.068577,36000000.0
3065971,22,2019-12-29 00:00:00,3.86001,597.535984,25.682372,36.65838,0.0,0.8,6.709571,0.068577,36000000.0
3065972,23,2019-12-29 00:00:00,3.86001,597.535984,25.682372,36.65838,0.0,0.8,6.709571,0.068577,36000000.0
3065973,24,2019-12-29 00:00:00,3.86001,597.535984,25.682372,36.65838,0.0,0.8,6.709571,0.068577,36000000.0


In [19]:
input_temp_df = pd.read_csv("./../../output/py_temp_total05.csv")

flattened_inp_temp = input_temp_df.iloc[:-1,1:].to_numpy().flatten() #this iloc is to remove the time column
flattened_out_temp = input_temp_df.iloc[1:,1:].to_numpy().flatten() #this iloc is to remove the time column
time_stamp = input_temp_df['time'][1:].repeat(depth_steps).values
data = {'time':time_stamp, 'input_temp':flattened_inp_temp, 'temp_total05':flattened_out_temp, 'depth':depth_list}
input_temp_df = pd.DataFrame(data=data)
input_temp_df 

,time,input_temp,temp_total05,depth
0,2006-01-01 02:00:00,5.431685,5.402835,1
1,2006-01-01 02:00:00,5.464454,5.429684,2
2,2006-01-01 02:00:00,5.481707,5.452260,3
3,2006-01-01 02:00:00,5.490656,5.452260,4
4,2006-01-01 02:00:00,5.495258,5.486623,5
...,...,...,...,...
3065970,2019-12-29 00:00:00,0.478190,0.496168,21
3065971,2019-12-29 00:00:00,0.461851,0.483696,22
3065972,2019-12-29 00:00:00,0.456712,0.471708,23
3065973,2019-12-29 00:00:00,0.456712,0.457236,24


In [20]:
depth_list.shape

(3065975,)

In [21]:
final_df = meterological_data_df.merge(input_temp_df, how='inner', on=['time','depth'])
final_df

,depth,time,AirTemp_degC,Longwave_Wm-2,Latent_Wm-2,Sensible_Wm-2,Shortwave_Wm-2,lightExtinct_m-1,ShearVelocity_mS-1,ShearStress_Nm-2,Area_m2,input_temp,temp_total05
0,1,2006-01-01 02:00:00,-1.220007,590.851292,-30.129462,-35.559056,0.0,0.8,1.757216,0.008038,36000000.0,5.431685,5.402835
1,2,2006-01-01 02:00:00,-1.220007,590.851292,-30.129462,-35.559056,0.0,0.8,1.757216,0.008038,36000000.0,5.464454,5.429684
2,3,2006-01-01 02:00:00,-1.220007,590.851292,-30.129462,-35.559056,0.0,0.8,1.757216,0.008038,36000000.0,5.481707,5.452260
3,4,2006-01-01 02:00:00,-1.220007,590.851292,-30.129462,-35.559056,0.0,0.8,1.757216,0.008038,36000000.0,5.490656,5.452260
4,5,2006-01-01 02:00:00,-1.220007,590.851292,-30.129462,-35.559056,0.0,0.8,1.757216,0.008038,36000000.0,5.495258,5.486623
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3065970,21,2019-12-29 00:00:00,3.86001,597.535984,25.682372,36.65838,0.0,0.8,6.709571,0.068577,36000000.0,0.478190,0.496168
3065971,22,2019-12-29 00:00:00,3.86001,597.535984,25.682372,36.65838,0.0,0.8,6.709571,0.068577,36000000.0,0.461851,0.483696
3065972,23,2019-12-29 00:00:00,3.86001,597.535984,25.682372,36.65838,0.0,0.8,6.709571,0.068577,36000000.0,0.456712,0.471708
3065973,24,2019-12-29 00:00:00,3.86001,597.535984,25.682372,36.65838,0.0,0.8,6.709571,0.068577,36000000.0,0.456712,0.457236


In [22]:
buoyancy_data_df = pd.read_csv("./../../output/py_buoyancy.csv")
#Removing Nans
#buoyancy_data_df['n2S-2_1'] = buoyancy_data_df['n2S-2_2']
#buoyancy_data_df['n2S-2_25'] = buoyancy_data_df['n2S-2_24']
buoyancy_data_df['24'] = buoyancy_data_df['23']

buoyancy_data_df = buoyancy_data_df[1:] # considering everything from 2nd time step

flattened_buoy = buoyancy_data_df.iloc[:,1:].to_numpy().flatten()
time_stamp = buoyancy_data_df['time'].repeat(depth_steps).values

data = {'time':time_stamp, 'buoyancy':flattened_buoy, 'depth':depth_list}
buoyancy_data_df = pd.DataFrame(data=data)

buoyancy_data_df

,time,buoyancy,depth
0,2006-01-01 02:00:00,-7.374820e-06,1
1,2006-01-01 02:00:00,-3.947632e-06,2
2,2006-01-01 02:00:00,-2.065015e-06,3
3,2006-01-01 02:00:00,-1.066679e-06,4
4,2006-01-01 02:00:00,-5.419944e-07,5
...,...,...,...
3065970,2019-12-29 00:00:00,-9.530742e-06,21
3065971,2019-12-29 00:00:00,-3.007825e-06,22
3065972,2019-12-29 00:00:00,0.000000e+00,23
3065973,2019-12-29 00:00:00,1.010495e-03,24


In [23]:
final_df = final_df.merge(buoyancy_data_df, how='inner', on=['time','depth'])
final_df

,depth,time,AirTemp_degC,Longwave_Wm-2,Latent_Wm-2,Sensible_Wm-2,Shortwave_Wm-2,lightExtinct_m-1,ShearVelocity_mS-1,ShearStress_Nm-2,Area_m2,input_temp,temp_total05,buoyancy
0,1,2006-01-01 02:00:00,-1.220007,590.851292,-30.129462,-35.559056,0.0,0.8,1.757216,0.008038,36000000.0,5.431685,5.402835,-7.374820e-06
1,2,2006-01-01 02:00:00,-1.220007,590.851292,-30.129462,-35.559056,0.0,0.8,1.757216,0.008038,36000000.0,5.464454,5.429684,-3.947632e-06
2,3,2006-01-01 02:00:00,-1.220007,590.851292,-30.129462,-35.559056,0.0,0.8,1.757216,0.008038,36000000.0,5.481707,5.452260,-2.065015e-06
3,4,2006-01-01 02:00:00,-1.220007,590.851292,-30.129462,-35.559056,0.0,0.8,1.757216,0.008038,36000000.0,5.490656,5.452260,-1.066679e-06
4,5,2006-01-01 02:00:00,-1.220007,590.851292,-30.129462,-35.559056,0.0,0.8,1.757216,0.008038,36000000.0,5.495258,5.486623,-5.419944e-07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3065970,21,2019-12-29 00:00:00,3.86001,597.535984,25.682372,36.65838,0.0,0.8,6.709571,0.068577,36000000.0,0.478190,0.496168,-9.530742e-06
3065971,22,2019-12-29 00:00:00,3.86001,597.535984,25.682372,36.65838,0.0,0.8,6.709571,0.068577,36000000.0,0.461851,0.483696,-3.007825e-06
3065972,23,2019-12-29 00:00:00,3.86001,597.535984,25.682372,36.65838,0.0,0.8,6.709571,0.068577,36000000.0,0.456712,0.471708,0.000000e+00
3065973,24,2019-12-29 00:00:00,3.86001,597.535984,25.682372,36.65838,0.0,0.8,6.709571,0.068577,36000000.0,0.456712,0.457236,1.010495e-03


In [24]:
out_diffusivity_df = pd.read_csv("./../../output/py_diff.csv")

out_diffusivity_df = out_diffusivity_df[1:] # considering everything from 2nd time step

flattened_diff = out_diffusivity_df.iloc[:,1:].to_numpy().flatten()
time_stamp = out_diffusivity_df['time'].repeat(depth_steps).values

data = {'time':time_stamp, 'diffusivity':flattened_diff, 'depth':depth_list}
out_diffusivity_df = pd.DataFrame(data=data)

out_diffusivity_df

,time,diffusivity,depth
0,2006-01-01 02:00:00,3.719017e-05,1
1,2006-01-01 02:00:00,3.719017e-05,2
2,2006-01-01 02:00:00,3.719017e-05,3
3,2006-01-01 02:00:00,3.719017e-05,4
4,2006-01-01 02:00:00,3.719017e-05,5
...,...,...,...
3065970,2019-12-29 00:00:00,6.358731e-07,21
3065971,2019-12-29 00:00:00,6.358731e-07,22
3065972,2019-12-29 00:00:00,6.358731e-07,23
3065973,2019-12-29 00:00:00,2.017497e-07,24


In [25]:
final_df = final_df.merge(out_diffusivity_df, how='inner', on=['time','depth'])
final_df

,depth,time,AirTemp_degC,Longwave_Wm-2,Latent_Wm-2,Sensible_Wm-2,Shortwave_Wm-2,lightExtinct_m-1,ShearVelocity_mS-1,ShearStress_Nm-2,Area_m2,input_temp,temp_total05,buoyancy,diffusivity
0,1,2006-01-01 02:00:00,-1.220007,590.851292,-30.129462,-35.559056,0.0,0.8,1.757216,0.008038,36000000.0,5.431685,5.402835,-7.374820e-06,3.719017e-05
1,2,2006-01-01 02:00:00,-1.220007,590.851292,-30.129462,-35.559056,0.0,0.8,1.757216,0.008038,36000000.0,5.464454,5.429684,-3.947632e-06,3.719017e-05
2,3,2006-01-01 02:00:00,-1.220007,590.851292,-30.129462,-35.559056,0.0,0.8,1.757216,0.008038,36000000.0,5.481707,5.452260,-2.065015e-06,3.719017e-05
3,4,2006-01-01 02:00:00,-1.220007,590.851292,-30.129462,-35.559056,0.0,0.8,1.757216,0.008038,36000000.0,5.490656,5.452260,-1.066679e-06,3.719017e-05
4,5,2006-01-01 02:00:00,-1.220007,590.851292,-30.129462,-35.559056,0.0,0.8,1.757216,0.008038,36000000.0,5.495258,5.486623,-5.419944e-07,3.719017e-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3065970,21,2019-12-29 00:00:00,3.86001,597.535984,25.682372,36.65838,0.0,0.8,6.709571,0.068577,36000000.0,0.478190,0.496168,-9.530742e-06,6.358731e-07
3065971,22,2019-12-29 00:00:00,3.86001,597.535984,25.682372,36.65838,0.0,0.8,6.709571,0.068577,36000000.0,0.461851,0.483696,-3.007825e-06,6.358731e-07
3065972,23,2019-12-29 00:00:00,3.86001,597.535984,25.682372,36.65838,0.0,0.8,6.709571,0.068577,36000000.0,0.456712,0.471708,0.000000e+00,6.358731e-07
3065973,24,2019-12-29 00:00:00,3.86001,597.535984,25.682372,36.65838,0.0,0.8,6.709571,0.068577,36000000.0,0.456712,0.457236,1.010495e-03,2.017497e-07


In [26]:
out_temp_df = pd.read_csv("./../../output/py_temp_heat01.csv")

out_temp_df = out_temp_df[1:] # considering everything from 2nd time step

flattened_temp = out_temp_df.iloc[:,1:].to_numpy().flatten()
time_stamp = out_temp_df['time'].repeat(depth_steps).values

data = {'time':time_stamp, 'temp_heat01':flattened_temp, 'depth':depth_list}
out_temp_df = pd.DataFrame(data=data)

out_temp_df

,time,temp_heat01,depth
0,2006-01-01 02:00:00,5.313505,1
1,2006-01-01 02:00:00,5.464459,2
2,2006-01-01 02:00:00,5.481712,3
3,2006-01-01 02:00:00,5.490660,4
4,2006-01-01 02:00:00,5.495260,5
...,...,...,...
3065970,2019-12-29 00:00:00,0.478225,21
3065971,2019-12-29 00:00:00,0.461898,22
3065972,2019-12-29 00:00:00,0.456792,23
3065973,2019-12-29 00:00:00,0.456712,24


In [27]:
final_df = final_df.merge(out_temp_df, how='inner', on=['time','depth'])
final_df

,depth,time,AirTemp_degC,Longwave_Wm-2,Latent_Wm-2,Sensible_Wm-2,Shortwave_Wm-2,lightExtinct_m-1,ShearVelocity_mS-1,ShearStress_Nm-2,Area_m2,input_temp,temp_total05,buoyancy,diffusivity,temp_heat01
0,1,2006-01-01 02:00:00,-1.220007,590.851292,-30.129462,-35.559056,0.0,0.8,1.757216,0.008038,36000000.0,5.431685,5.402835,-7.374820e-06,3.719017e-05,5.313505
1,2,2006-01-01 02:00:00,-1.220007,590.851292,-30.129462,-35.559056,0.0,0.8,1.757216,0.008038,36000000.0,5.464454,5.429684,-3.947632e-06,3.719017e-05,5.464459
2,3,2006-01-01 02:00:00,-1.220007,590.851292,-30.129462,-35.559056,0.0,0.8,1.757216,0.008038,36000000.0,5.481707,5.452260,-2.065015e-06,3.719017e-05,5.481712
3,4,2006-01-01 02:00:00,-1.220007,590.851292,-30.129462,-35.559056,0.0,0.8,1.757216,0.008038,36000000.0,5.490656,5.452260,-1.066679e-06,3.719017e-05,5.490660
4,5,2006-01-01 02:00:00,-1.220007,590.851292,-30.129462,-35.559056,0.0,0.8,1.757216,0.008038,36000000.0,5.495258,5.486623,-5.419944e-07,3.719017e-05,5.495260
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3065970,21,2019-12-29 00:00:00,3.86001,597.535984,25.682372,36.65838,0.0,0.8,6.709571,0.068577,36000000.0,0.478190,0.496168,-9.530742e-06,6.358731e-07,0.478225
3065971,22,2019-12-29 00:00:00,3.86001,597.535984,25.682372,36.65838,0.0,0.8,6.709571,0.068577,36000000.0,0.461851,0.483696,-3.007825e-06,6.358731e-07,0.461898
3065972,23,2019-12-29 00:00:00,3.86001,597.535984,25.682372,36.65838,0.0,0.8,6.709571,0.068577,36000000.0,0.456712,0.471708,0.000000e+00,6.358731e-07,0.456792
3065973,24,2019-12-29 00:00:00,3.86001,597.535984,25.682372,36.65838,0.0,0.8,6.709571,0.068577,36000000.0,0.456712,0.457236,1.010495e-03,2.017497e-07,0.456712


In [28]:
out_temp_df = pd.read_csv("./../../output/py_temp_diff02.csv")

out_temp_df = out_temp_df[1:] # considering everything from 2nd time step

flattened_temp = out_temp_df.iloc[:,1:].to_numpy().flatten()
time_stamp = out_temp_df['time'].repeat(depth_steps).values

data = {'time':time_stamp, 'temp_diff02':flattened_temp, 'depth':depth_list}
out_temp_df = pd.DataFrame(data=data)

out_temp_df

,time,temp_diff02,depth
0,2006-01-01 02:00:00,5.313505,1
1,2006-01-01 02:00:00,5.448556,2
2,2006-01-01 02:00:00,5.479754,3
3,2006-01-01 02:00:00,5.490013,4
4,2006-01-01 02:00:00,5.494945,5
...,...,...,...
3065970,2019-12-29 00:00:00,0.478225,21
3065971,2019-12-29 00:00:00,0.461923,22
3065972,2019-12-29 00:00:00,0.456284,23
3065973,2019-12-29 00:00:00,0.001672,24


In [29]:
final_df = final_df.merge(out_temp_df, how='inner', on=['time','depth'])
final_df

,depth,time,AirTemp_degC,Longwave_Wm-2,Latent_Wm-2,Sensible_Wm-2,Shortwave_Wm-2,lightExtinct_m-1,ShearVelocity_mS-1,ShearStress_Nm-2,Area_m2,input_temp,temp_total05,buoyancy,diffusivity,temp_heat01,temp_diff02
0,1,2006-01-01 02:00:00,-1.220007,590.851292,-30.129462,-35.559056,0.0,0.8,1.757216,0.008038,36000000.0,5.431685,5.402835,-7.374820e-06,3.719017e-05,5.313505,5.313505
1,2,2006-01-01 02:00:00,-1.220007,590.851292,-30.129462,-35.559056,0.0,0.8,1.757216,0.008038,36000000.0,5.464454,5.429684,-3.947632e-06,3.719017e-05,5.464459,5.448556
2,3,2006-01-01 02:00:00,-1.220007,590.851292,-30.129462,-35.559056,0.0,0.8,1.757216,0.008038,36000000.0,5.481707,5.452260,-2.065015e-06,3.719017e-05,5.481712,5.479754
3,4,2006-01-01 02:00:00,-1.220007,590.851292,-30.129462,-35.559056,0.0,0.8,1.757216,0.008038,36000000.0,5.490656,5.452260,-1.066679e-06,3.719017e-05,5.490660,5.490013
4,5,2006-01-01 02:00:00,-1.220007,590.851292,-30.129462,-35.559056,0.0,0.8,1.757216,0.008038,36000000.0,5.495258,5.486623,-5.419944e-07,3.719017e-05,5.495260,5.494945
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3065970,21,2019-12-29 00:00:00,3.86001,597.535984,25.682372,36.65838,0.0,0.8,6.709571,0.068577,36000000.0,0.478190,0.496168,-9.530742e-06,6.358731e-07,0.478225,0.478225
3065971,22,2019-12-29 00:00:00,3.86001,597.535984,25.682372,36.65838,0.0,0.8,6.709571,0.068577,36000000.0,0.461851,0.483696,-3.007825e-06,6.358731e-07,0.461898,0.461923
3065972,23,2019-12-29 00:00:00,3.86001,597.535984,25.682372,36.65838,0.0,0.8,6.709571,0.068577,36000000.0,0.456712,0.471708,0.000000e+00,6.358731e-07,0.456792,0.456284
3065973,24,2019-12-29 00:00:00,3.86001,597.535984,25.682372,36.65838,0.0,0.8,6.709571,0.068577,36000000.0,0.456712,0.457236,1.010495e-03,2.017497e-07,0.456712,0.001672


In [30]:
datetime_list =[datetime.datetime.strptime(date, '%Y-%m-%d %H:%M:%S') for date in final_df['time']]
day_of_year_list = [t.timetuple().tm_yday for t in datetime_list]
time_of_day_list = [t.hour for t in datetime_list]

In [31]:
final_df['day_of_year']=day_of_year_list
final_df['time_of_day']=time_of_day_list

In [32]:
final_df

,depth,time,AirTemp_degC,Longwave_Wm-2,Latent_Wm-2,Sensible_Wm-2,Shortwave_Wm-2,lightExtinct_m-1,ShearVelocity_mS-1,ShearStress_Nm-2,Area_m2,input_temp,temp_total05,buoyancy,diffusivity,temp_heat01,temp_diff02,day_of_year,time_of_day
0,1,2006-01-01 02:00:00,-1.220007,590.851292,-30.129462,-35.559056,0.0,0.8,1.757216,0.008038,36000000.0,5.431685,5.402835,-7.374820e-06,3.719017e-05,5.313505,5.313505,1,2
1,2,2006-01-01 02:00:00,-1.220007,590.851292,-30.129462,-35.559056,0.0,0.8,1.757216,0.008038,36000000.0,5.464454,5.429684,-3.947632e-06,3.719017e-05,5.464459,5.448556,1,2
2,3,2006-01-01 02:00:00,-1.220007,590.851292,-30.129462,-35.559056,0.0,0.8,1.757216,0.008038,36000000.0,5.481707,5.452260,-2.065015e-06,3.719017e-05,5.481712,5.479754,1,2
3,4,2006-01-01 02:00:00,-1.220007,590.851292,-30.129462,-35.559056,0.0,0.8,1.757216,0.008038,36000000.0,5.490656,5.452260,-1.066679e-06,3.719017e-05,5.490660,5.490013,1,2
4,5,2006-01-01 02:00:00,-1.220007,590.851292,-30.129462,-35.559056,0.0,0.8,1.757216,0.008038,36000000.0,5.495258,5.486623,-5.419944e-07,3.719017e-05,5.495260,5.494945,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3065970,21,2019-12-29 00:00:00,3.86001,597.535984,25.682372,36.65838,0.0,0.8,6.709571,0.068577,36000000.0,0.478190,0.496168,-9.530742e-06,6.358731e-07,0.478225,0.478225,363,0
3065971,22,2019-12-29 00:00:00,3.86001,597.535984,25.682372,36.65838,0.0,0.8,6.709571,0.068577,36000000.0,0.461851,0.483696,-3.007825e-06,6.358731e-07,0.461898,0.461923,363,0
3065972,23,2019-12-29 00:00:00,3.86001,597.535984,25.682372,36.65838,0.0,0.8,6.709571,0.068577,36000000.0,0.456712,0.471708,0.000000e+00,6.358731e-07,0.456792,0.456284,363,0
3065973,24,2019-12-29 00:00:00,3.86001,597.535984,25.682372,36.65838,0.0,0.8,6.709571,0.068577,36000000.0,0.456712,0.457236,1.010495e-03,2.017497e-07,0.456712,0.001672,363,0


In [33]:
temp_mix03_df = pd.read_csv("./../../output/py_temp_mix03.csv")

temp_mix03_df = temp_mix03_df[1:] # considering everything from 2nd time step

flattened_temp = temp_mix03_df.iloc[:,1:].to_numpy().flatten()
time_stamp = temp_mix03_df['time'].repeat(depth_steps).values

data = {'time':time_stamp, 'temp_mix03':flattened_temp, 'depth':depth_list}
temp_mix03_df = pd.DataFrame(data=data)

temp_mix03_df

,time,temp_mix03,depth
0,2006-01-01 02:00:00,5.379096,1
1,2006-01-01 02:00:00,5.427976,2
2,2006-01-01 02:00:00,5.458125,3
3,2006-01-01 02:00:00,5.476137,4
4,2006-01-01 02:00:00,5.486623,5
...,...,...,...
3065970,2019-12-29 00:00:00,0.478225,21
3065971,2019-12-29 00:00:00,0.461923,22
3065972,2019-12-29 00:00:00,0.456284,23
3065973,2019-12-29 00:00:00,0.001672,24


In [34]:
final_df = final_df.merge(temp_mix03_df, how='inner', on=['time','depth'])
final_df

,depth,time,AirTemp_degC,Longwave_Wm-2,Latent_Wm-2,Sensible_Wm-2,Shortwave_Wm-2,lightExtinct_m-1,ShearVelocity_mS-1,ShearStress_Nm-2,Area_m2,input_temp,temp_total05,buoyancy,diffusivity,temp_heat01,temp_diff02,day_of_year,time_of_day,temp_mix03
0,1,2006-01-01 02:00:00,-1.220007,590.851292,-30.129462,-35.559056,0.0,0.8,1.757216,0.008038,36000000.0,5.431685,5.402835,-7.374820e-06,3.719017e-05,5.313505,5.313505,1,2,5.379096
1,2,2006-01-01 02:00:00,-1.220007,590.851292,-30.129462,-35.559056,0.0,0.8,1.757216,0.008038,36000000.0,5.464454,5.429684,-3.947632e-06,3.719017e-05,5.464459,5.448556,1,2,5.427976
2,3,2006-01-01 02:00:00,-1.220007,590.851292,-30.129462,-35.559056,0.0,0.8,1.757216,0.008038,36000000.0,5.481707,5.452260,-2.065015e-06,3.719017e-05,5.481712,5.479754,1,2,5.458125
3,4,2006-01-01 02:00:00,-1.220007,590.851292,-30.129462,-35.559056,0.0,0.8,1.757216,0.008038,36000000.0,5.490656,5.452260,-1.066679e-06,3.719017e-05,5.490660,5.490013,1,2,5.476137
4,5,2006-01-01 02:00:00,-1.220007,590.851292,-30.129462,-35.559056,0.0,0.8,1.757216,0.008038,36000000.0,5.495258,5.486623,-5.419944e-07,3.719017e-05,5.495260,5.494945,1,2,5.486623
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3065970,21,2019-12-29 00:00:00,3.86001,597.535984,25.682372,36.65838,0.0,0.8,6.709571,0.068577,36000000.0,0.478190,0.496168,-9.530742e-06,6.358731e-07,0.478225,0.478225,363,0,0.478225
3065971,22,2019-12-29 00:00:00,3.86001,597.535984,25.682372,36.65838,0.0,0.8,6.709571,0.068577,36000000.0,0.461851,0.483696,-3.007825e-06,6.358731e-07,0.461898,0.461923,363,0,0.461923
3065972,23,2019-12-29 00:00:00,3.86001,597.535984,25.682372,36.65838,0.0,0.8,6.709571,0.068577,36000000.0,0.456712,0.471708,0.000000e+00,6.358731e-07,0.456792,0.456284,363,0,0.456284
3065973,24,2019-12-29 00:00:00,3.86001,597.535984,25.682372,36.65838,0.0,0.8,6.709571,0.068577,36000000.0,0.456712,0.457236,1.010495e-03,2.017497e-07,0.456712,0.001672,363,0,0.001672


In [35]:
temp_conv03_df = pd.read_csv("./../../output/py_temp_conv04.csv")

temp_conv03_df = temp_conv03_df[1:] # considering everything from 2nd time step

flattened_temp = temp_conv03_df.iloc[:,1:].to_numpy().flatten()
time_stamp = temp_conv03_df['time'].repeat(depth_steps).values

data = {'time':time_stamp, 'temp_conv04':flattened_temp, 'depth':depth_list}
temp_conv03_df = pd.DataFrame(data=data)

temp_conv03_df

,time,temp_conv04,depth
0,2006-01-01 02:00:00,5.402835,1
1,2006-01-01 02:00:00,5.429684,2
2,2006-01-01 02:00:00,5.452260,3
3,2006-01-01 02:00:00,5.452260,4
4,2006-01-01 02:00:00,5.486623,5
...,...,...,...
3065970,2019-12-29 00:00:00,0.496168,21
3065971,2019-12-29 00:00:00,0.483696,22
3065972,2019-12-29 00:00:00,0.471708,23
3065973,2019-12-29 00:00:00,0.457236,24


In [36]:
final_df = final_df.merge(temp_conv03_df, how='inner', on=['time','depth'])
final_df

,depth,time,AirTemp_degC,Longwave_Wm-2,Latent_Wm-2,Sensible_Wm-2,Shortwave_Wm-2,lightExtinct_m-1,ShearVelocity_mS-1,ShearStress_Nm-2,...,input_temp,temp_total05,buoyancy,diffusivity,temp_heat01,temp_diff02,day_of_year,time_of_day,temp_mix03,temp_conv04
0,1,2006-01-01 02:00:00,-1.220007,590.851292,-30.129462,-35.559056,0.0,0.8,1.757216,0.008038,...,5.431685,5.402835,-7.374820e-06,3.719017e-05,5.313505,5.313505,1,2,5.379096,5.402835
1,2,2006-01-01 02:00:00,-1.220007,590.851292,-30.129462,-35.559056,0.0,0.8,1.757216,0.008038,...,5.464454,5.429684,-3.947632e-06,3.719017e-05,5.464459,5.448556,1,2,5.427976,5.429684
2,3,2006-01-01 02:00:00,-1.220007,590.851292,-30.129462,-35.559056,0.0,0.8,1.757216,0.008038,...,5.481707,5.452260,-2.065015e-06,3.719017e-05,5.481712,5.479754,1,2,5.458125,5.452260
3,4,2006-01-01 02:00:00,-1.220007,590.851292,-30.129462,-35.559056,0.0,0.8,1.757216,0.008038,...,5.490656,5.452260,-1.066679e-06,3.719017e-05,5.490660,5.490013,1,2,5.476137,5.452260
4,5,2006-01-01 02:00:00,-1.220007,590.851292,-30.129462,-35.559056,0.0,0.8,1.757216,0.008038,...,5.495258,5.486623,-5.419944e-07,3.719017e-05,5.495260,5.494945,1,2,5.486623,5.486623
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3065970,21,2019-12-29 00:00:00,3.86001,597.535984,25.682372,36.65838,0.0,0.8,6.709571,0.068577,...,0.478190,0.496168,-9.530742e-06,6.358731e-07,0.478225,0.478225,363,0,0.478225,0.496168
3065971,22,2019-12-29 00:00:00,3.86001,597.535984,25.682372,36.65838,0.0,0.8,6.709571,0.068577,...,0.461851,0.483696,-3.007825e-06,6.358731e-07,0.461898,0.461923,363,0,0.461923,0.483696
3065972,23,2019-12-29 00:00:00,3.86001,597.535984,25.682372,36.65838,0.0,0.8,6.709571,0.068577,...,0.456712,0.471708,0.000000e+00,6.358731e-07,0.456792,0.456284,363,0,0.456284,0.471708
3065973,24,2019-12-29 00:00:00,3.86001,597.535984,25.682372,36.65838,0.0,0.8,6.709571,0.068577,...,0.456712,0.457236,1.010495e-03,2.017497e-07,0.456712,0.001672,363,0,0.001672,0.457236


In [37]:
temp_initial00_df = pd.read_csv("./../../output/py_temp_initial00.csv")

temp_initial00_df = temp_initial00_df[1:] # considering everything from 2nd time step

flattened_temp = temp_initial00_df.iloc[:,1:].to_numpy().flatten()
time_stamp = temp_initial00_df['time'].repeat(depth_steps).values

data = {'time':time_stamp, 'temp_initial00':flattened_temp, 'depth':depth_list}
temp_initial00_df = pd.DataFrame(data=data)

temp_initial00_df

,time,temp_initial00,depth
0,2006-01-01 02:00:00,5.431685,1
1,2006-01-01 02:00:00,5.464454,2
2,2006-01-01 02:00:00,5.481707,3
3,2006-01-01 02:00:00,5.490656,4
4,2006-01-01 02:00:00,5.495258,5
...,...,...,...
3065970,2019-12-29 00:00:00,0.478190,21
3065971,2019-12-29 00:00:00,0.461851,22
3065972,2019-12-29 00:00:00,0.456712,23
3065973,2019-12-29 00:00:00,0.456712,24


In [38]:
final_df = final_df.merge(temp_initial00_df, how='inner', on=['time','depth'])
final_df

,depth,time,AirTemp_degC,Longwave_Wm-2,Latent_Wm-2,Sensible_Wm-2,Shortwave_Wm-2,lightExtinct_m-1,ShearVelocity_mS-1,ShearStress_Nm-2,...,temp_total05,buoyancy,diffusivity,temp_heat01,temp_diff02,day_of_year,time_of_day,temp_mix03,temp_conv04,temp_initial00
0,1,2006-01-01 02:00:00,-1.220007,590.851292,-30.129462,-35.559056,0.0,0.8,1.757216,0.008038,...,5.402835,-7.374820e-06,3.719017e-05,5.313505,5.313505,1,2,5.379096,5.402835,5.431685
1,2,2006-01-01 02:00:00,-1.220007,590.851292,-30.129462,-35.559056,0.0,0.8,1.757216,0.008038,...,5.429684,-3.947632e-06,3.719017e-05,5.464459,5.448556,1,2,5.427976,5.429684,5.464454
2,3,2006-01-01 02:00:00,-1.220007,590.851292,-30.129462,-35.559056,0.0,0.8,1.757216,0.008038,...,5.452260,-2.065015e-06,3.719017e-05,5.481712,5.479754,1,2,5.458125,5.452260,5.481707
3,4,2006-01-01 02:00:00,-1.220007,590.851292,-30.129462,-35.559056,0.0,0.8,1.757216,0.008038,...,5.452260,-1.066679e-06,3.719017e-05,5.490660,5.490013,1,2,5.476137,5.452260,5.490656
4,5,2006-01-01 02:00:00,-1.220007,590.851292,-30.129462,-35.559056,0.0,0.8,1.757216,0.008038,...,5.486623,-5.419944e-07,3.719017e-05,5.495260,5.494945,1,2,5.486623,5.486623,5.495258
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3065970,21,2019-12-29 00:00:00,3.86001,597.535984,25.682372,36.65838,0.0,0.8,6.709571,0.068577,...,0.496168,-9.530742e-06,6.358731e-07,0.478225,0.478225,363,0,0.478225,0.496168,0.478190
3065971,22,2019-12-29 00:00:00,3.86001,597.535984,25.682372,36.65838,0.0,0.8,6.709571,0.068577,...,0.483696,-3.007825e-06,6.358731e-07,0.461898,0.461923,363,0,0.461923,0.483696,0.461851
3065972,23,2019-12-29 00:00:00,3.86001,597.535984,25.682372,36.65838,0.0,0.8,6.709571,0.068577,...,0.471708,0.000000e+00,6.358731e-07,0.456792,0.456284,363,0,0.456284,0.471708,0.456712
3065973,24,2019-12-29 00:00:00,3.86001,597.535984,25.682372,36.65838,0.0,0.8,6.709571,0.068577,...,0.457236,1.010495e-03,2.017497e-07,0.456712,0.001672,363,0,0.001672,0.457236,0.456712


In [39]:
temp_obs_df = pd.read_csv("./../../output/py_observed_temp.csv")

temp_obs_df = temp_obs_df[1:] # considering everything from 2nd time step

flattened_temp = temp_obs_df.iloc[:,1:].to_numpy().flatten()
time_stamp = temp_obs_df['time'].repeat(depth_steps).values

data = {'time':time_stamp, 'obs_temp':flattened_temp, 'depth':depth_list}
temp_obs_df = pd.DataFrame(data=data)

temp_obs_df

print(flattened_temp.shape)

print(time_stamp.shape)

print(depth_list.shape)

(3065975,)
(3065975,)
(3065975,)


In [40]:
final_df = final_df.merge(temp_obs_df, how='inner', on=['time','depth'])
final_df

,depth,time,AirTemp_degC,Longwave_Wm-2,Latent_Wm-2,Sensible_Wm-2,Shortwave_Wm-2,lightExtinct_m-1,ShearVelocity_mS-1,ShearStress_Nm-2,...,buoyancy,diffusivity,temp_heat01,temp_diff02,day_of_year,time_of_day,temp_mix03,temp_conv04,temp_initial00,obs_temp
0,1,2006-01-01 02:00:00,-1.220007,590.851292,-30.129462,-35.559056,0.0,0.8,1.757216,0.008038,...,-7.374820e-06,3.719017e-05,5.313505,5.313505,1,2,5.379096,5.402835,5.431685,NaN
1,2,2006-01-01 02:00:00,-1.220007,590.851292,-30.129462,-35.559056,0.0,0.8,1.757216,0.008038,...,-3.947632e-06,3.719017e-05,5.464459,5.448556,1,2,5.427976,5.429684,5.464454,NaN
2,3,2006-01-01 02:00:00,-1.220007,590.851292,-30.129462,-35.559056,0.0,0.8,1.757216,0.008038,...,-2.065015e-06,3.719017e-05,5.481712,5.479754,1,2,5.458125,5.452260,5.481707,NaN
3,4,2006-01-01 02:00:00,-1.220007,590.851292,-30.129462,-35.559056,0.0,0.8,1.757216,0.008038,...,-1.066679e-06,3.719017e-05,5.490660,5.490013,1,2,5.476137,5.452260,5.490656,NaN
4,5,2006-01-01 02:00:00,-1.220007,590.851292,-30.129462,-35.559056,0.0,0.8,1.757216,0.008038,...,-5.419944e-07,3.719017e-05,5.495260,5.494945,1,2,5.486623,5.486623,5.495258,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3065970,21,2019-12-29 00:00:00,3.86001,597.535984,25.682372,36.65838,0.0,0.8,6.709571,0.068577,...,-9.530742e-06,6.358731e-07,0.478225,0.478225,363,0,0.478225,0.496168,0.478190,NaN
3065971,22,2019-12-29 00:00:00,3.86001,597.535984,25.682372,36.65838,0.0,0.8,6.709571,0.068577,...,-3.007825e-06,6.358731e-07,0.461898,0.461923,363,0,0.461923,0.483696,0.461851,NaN
3065972,23,2019-12-29 00:00:00,3.86001,597.535984,25.682372,36.65838,0.0,0.8,6.709571,0.068577,...,0.000000e+00,6.358731e-07,0.456792,0.456284,363,0,0.456284,0.471708,0.456712,NaN
3065973,24,2019-12-29 00:00:00,3.86001,597.535984,25.682372,36.65838,0.0,0.8,6.709571,0.068577,...,1.010495e-03,2.017497e-07,0.456712,0.001672,363,0,0.001672,0.457236,0.456712,NaN


In [41]:

#temp_obs_df2 = pd.read_csv("./../output/observed_temp.csv")
#temp_obs_df = pd.read_csv("./../output/observed_temp.csv")


#print(flattened_inp_temp_obs.shape)

#temp_obs_df2 = temp_obs_df2[:-1] # considering everything from 2nd time step

#flattened_inp_temp_obs = temp_obs_df2.iloc[:,1:].to_numpy().flatten()

#time_stamp = input_temp_df['time'][1:].repeat(depth_steps).values

#data = {'time':time_stamp, 'obs_input':flattened_inp_temp_obs, 'depth':depth_list}
#temp_obs_df2 = pd.DataFrame(data=data)

#temp_obs_df2


temp_obs_df = pd.read_csv("./../../output/py_observed_temp.csv")

flattened_inp_temp = temp_obs_df.iloc[:-1,1:].to_numpy().flatten() #this iloc is to remove the time column
print(flattened_inp_temp.shape)

time_stamp = temp_obs_df['time'][1:].repeat(depth_steps).values
print(time_stamp.shape)

print(depth_list.shape)
data = {'time':time_stamp, 'input_obs':flattened_inp_temp,'depth':depth_list}
temp_obs_df = pd.DataFrame(data=data)
temp_obs_df 

(3065975,)
(3065975,)
(3065975,)


,time,input_obs,depth
0,2006-01-01 02:00:00,NaN,1
1,2006-01-01 02:00:00,NaN,2
2,2006-01-01 02:00:00,NaN,3
3,2006-01-01 02:00:00,NaN,4
4,2006-01-01 02:00:00,NaN,5
...,...,...,...
3065970,2019-12-29 00:00:00,NaN,21
3065971,2019-12-29 00:00:00,NaN,22
3065972,2019-12-29 00:00:00,NaN,23
3065973,2019-12-29 00:00:00,NaN,24


In [42]:
final_df = final_df.merge(temp_obs_df, how='inner', on=['time','depth'])
final_df

,depth,time,AirTemp_degC,Longwave_Wm-2,Latent_Wm-2,Sensible_Wm-2,Shortwave_Wm-2,lightExtinct_m-1,ShearVelocity_mS-1,ShearStress_Nm-2,...,diffusivity,temp_heat01,temp_diff02,day_of_year,time_of_day,temp_mix03,temp_conv04,temp_initial00,obs_temp,input_obs
0,1,2006-01-01 02:00:00,-1.220007,590.851292,-30.129462,-35.559056,0.0,0.8,1.757216,0.008038,...,3.719017e-05,5.313505,5.313505,1,2,5.379096,5.402835,5.431685,NaN,NaN
1,2,2006-01-01 02:00:00,-1.220007,590.851292,-30.129462,-35.559056,0.0,0.8,1.757216,0.008038,...,3.719017e-05,5.464459,5.448556,1,2,5.427976,5.429684,5.464454,NaN,NaN
2,3,2006-01-01 02:00:00,-1.220007,590.851292,-30.129462,-35.559056,0.0,0.8,1.757216,0.008038,...,3.719017e-05,5.481712,5.479754,1,2,5.458125,5.452260,5.481707,NaN,NaN
3,4,2006-01-01 02:00:00,-1.220007,590.851292,-30.129462,-35.559056,0.0,0.8,1.757216,0.008038,...,3.719017e-05,5.490660,5.490013,1,2,5.476137,5.452260,5.490656,NaN,NaN
4,5,2006-01-01 02:00:00,-1.220007,590.851292,-30.129462,-35.559056,0.0,0.8,1.757216,0.008038,...,3.719017e-05,5.495260,5.494945,1,2,5.486623,5.486623,5.495258,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3065970,21,2019-12-29 00:00:00,3.86001,597.535984,25.682372,36.65838,0.0,0.8,6.709571,0.068577,...,6.358731e-07,0.478225,0.478225,363,0,0.478225,0.496168,0.478190,NaN,NaN
3065971,22,2019-12-29 00:00:00,3.86001,597.535984,25.682372,36.65838,0.0,0.8,6.709571,0.068577,...,6.358731e-07,0.461898,0.461923,363,0,0.461923,0.483696,0.461851,NaN,NaN
3065972,23,2019-12-29 00:00:00,3.86001,597.535984,25.682372,36.65838,0.0,0.8,6.709571,0.068577,...,6.358731e-07,0.456792,0.456284,363,0,0.456284,0.471708,0.456712,NaN,NaN
3065973,24,2019-12-29 00:00:00,3.86001,597.535984,25.682372,36.65838,0.0,0.8,6.709571,0.068577,...,2.017497e-07,0.456712,0.001672,363,0,0.001672,0.457236,0.456712,NaN,NaN


In [43]:
ice_data_df = pd.read_csv("./../../output/py_icesnow.csv")
ice_data_df = ice_data_df[1:] # considering everything from 2nd time step

num_time_steps = ice_data_df.shape[0]
depth_list = np.array(list(range(1, depth_steps+1)) * num_time_steps)
depth_df = pd.DataFrame(data={'depth':depth_list})

#repeating the dataframe depth_steps number of times
ice_data_df = pd.DataFrame(np.repeat(ice_data_df.values, depth_steps, axis=0), columns=ice_data_df.columns)
ice_data_df = pd.concat([depth_df, ice_data_df], ignore_index=False, axis=1)
print(ice_data_df)

final_df = final_df.merge(ice_data_df, how='inner', on=['time','depth'])
final_df

         depth                 time       ice snow snowice
0            1  2006-01-01 02:00:00       0.0  0.0     0.0
1            2  2006-01-01 02:00:00       0.0  0.0     0.0
2            3  2006-01-01 02:00:00       0.0  0.0     0.0
3            4  2006-01-01 02:00:00       0.0  0.0     0.0
4            5  2006-01-01 02:00:00       0.0  0.0     0.0
...        ...                  ...       ...  ...     ...
3065970     21  2019-12-29 00:00:00  0.255083  0.0     0.0
3065971     22  2019-12-29 00:00:00  0.255083  0.0     0.0
3065972     23  2019-12-29 00:00:00  0.255083  0.0     0.0
3065973     24  2019-12-29 00:00:00  0.255083  0.0     0.0
3065974     25  2019-12-29 00:00:00  0.255083  0.0     0.0

[3065975 rows x 5 columns]


,depth,time,AirTemp_degC,Longwave_Wm-2,Latent_Wm-2,Sensible_Wm-2,Shortwave_Wm-2,lightExtinct_m-1,ShearVelocity_mS-1,ShearStress_Nm-2,...,day_of_year,time_of_day,temp_mix03,temp_conv04,temp_initial00,obs_temp,input_obs,ice,snow,snowice
0,1,2006-01-01 02:00:00,-1.220007,590.851292,-30.129462,-35.559056,0.0,0.8,1.757216,0.008038,...,1,2,5.379096,5.402835,5.431685,NaN,NaN,0.0,0.0,0.0
1,2,2006-01-01 02:00:00,-1.220007,590.851292,-30.129462,-35.559056,0.0,0.8,1.757216,0.008038,...,1,2,5.427976,5.429684,5.464454,NaN,NaN,0.0,0.0,0.0
2,3,2006-01-01 02:00:00,-1.220007,590.851292,-30.129462,-35.559056,0.0,0.8,1.757216,0.008038,...,1,2,5.458125,5.452260,5.481707,NaN,NaN,0.0,0.0,0.0
3,4,2006-01-01 02:00:00,-1.220007,590.851292,-30.129462,-35.559056,0.0,0.8,1.757216,0.008038,...,1,2,5.476137,5.452260,5.490656,NaN,NaN,0.0,0.0,0.0
4,5,2006-01-01 02:00:00,-1.220007,590.851292,-30.129462,-35.559056,0.0,0.8,1.757216,0.008038,...,1,2,5.486623,5.486623,5.495258,NaN,NaN,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3065970,21,2019-12-29 00:00:00,3.86001,597.535984,25.682372,36.65838,0.0,0.8,6.709571,0.068577,...,363,0,0.478225,0.496168,0.478190,NaN,NaN,0.255083,0.0,0.0
3065971,22,2019-12-29 00:00:00,3.86001,597.535984,25.682372,36.65838,0.0,0.8,6.709571,0.068577,...,363,0,0.461923,0.483696,0.461851,NaN,NaN,0.255083,0.0,0.0
3065972,23,2019-12-29 00:00:00,3.86001,597.535984,25.682372,36.65838,0.0,0.8,6.709571,0.068577,...,363,0,0.456284,0.471708,0.456712,NaN,NaN,0.255083,0.0,0.0
3065973,24,2019-12-29 00:00:00,3.86001,597.535984,25.682372,36.65838,0.0,0.8,6.709571,0.068577,...,363,0,0.001672,0.457236,0.456712,NaN,NaN,0.255083,0.0,0.0


In [44]:
final_df.to_csv("./../02_training/all_data_lake_modeling_in_time.csv", index=False)

In [45]:
final_df.to_csv("all_data_lake_modeling_in_time.csv", index=False)